#### zip 函数将名/值列表链在一起

In [1]:
keys = ["name", "age", "pay", "job"]
values = ["L.J", "35", "35000000$", "basketball player"] 
list(zip(keys, values))

[('name', 'L.J'),
 ('age', '35'),
 ('pay', '35000000$'),
 ('job', 'basketball player')]

In [2]:
james = dict(zip(keys, values))
james

{'name': 'L.J', 'age': '35', 'pay': '35000000$', 'job': 'basketball player'}

#### 可以通过一个健序列和所有健的可选初始值创建字典(便于初始化空字典)

In [3]:
fields = {"name", "age", "job"}
record = dict.fromkeys(fields, "?")
record

{'age': '?', 'name': '?', 'job': '?'}

#### 可以使字典本身表示数据库本身，即外层的字典是数据库，内层嵌套的字典式数据库中的记录

In [4]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

In [5]:
db = {}                 # 数据库
db["bob"] = bob         # 引用字典的字典
db["sue"] = sue

In [6]:
db["bob"]["name"]

'Bob Smith'

In [7]:
db["sue"]["pay"] * 1.1  # 涨薪

44000.0

In [8]:
# 这样的结构可以直接访问一条记录，而不必循环中寻找到它。
import pprint
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000}}


In [9]:
# 迭代
x = [db[key]["name"] for key in db]
y = [rec["name"] for rec in db.values()]
print(x, y)

['Bob Smith', 'Sue Jones'] ['Bob Smith', 'Sue Jones']


In [10]:
# 增
db["tom"] = dict(name="Tom", age=50, job=None, pay=0)
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 50, 'job': None, 'name': 'Tom', 'pay': 0}}


In [11]:
# 类似 SQL 查询
[rec["name"] for rec in db.values() if rec["age"] >= 45] 

['Sue Jones', 'Tom']

#### 使用pickle将记录保存更新进一个普通文件

In [12]:
# 保存
import pickle
with open("people_pickle", "wb") as dbfile:  # 使用 3. 
    pickle.dump(db, dbfile)                # db 可以单独的以初始化模块文件来组织

In [13]:
# 读取
with open("people_pickle", "rb") as dbfile:  # 使用 3. 
     db_ = pickle.load(dbfile)                
     pprint.pprint(db_)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 50, 'job': None, 'name': 'Tom', 'pay': 0}}


#### 当处理非常大的数据时，上述方法会变得很慢，因为更新一条数据就要重新读出和写入整个数据库，我们可以通过存储数据库中的一条记录到一个普通的文件的方式来改进

In [14]:
# 1  将每条记录存到对应的文件中
for (key, record) in [("bob", bob), ("sue", sue)]:
    with open(key+".pkl", "wb") as recfile:
        pickle.dump(record, recfile)

In [15]:
# 2  使用glob模块获取整个数据库。它根据文件拓展名来获取这个目录下所有该拓展名的文件。如果需要载入单个记录，可以打开对应的文件并使用pickle反序列化。为了得到一条记录，我们只需要载入一个记录而不需要整个数据库
import glob

# 查看整个数据库
for filename in glob.glob("*.pkl"):    
    with open(filename, "rb") as recfile:
        record = pickle.load(recfile)
        print(filename, "=>\n",record)

#　查看单个记录
suefile = open("sue.pkl", "rb")
print("sue.pkl['name']", "=>\n", pickle.load(suefile)["name"])
suefile.close()

bob.pkl =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue.pkl =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}
sue.pkl['name'] =>
 Sue Jones


In [16]:
# 3  当更新数据库，从对应文件抓取一条记录，在内存中进行修改，然后重新写入它的pickle文件，不用为了更新某段记录而对整个数据库进行操作

suefile = open("sue.pkl", "rb")
sue = pickle.load(suefile)
suefile.close()

sue["pay"] *= 1.10  # 给sue涨薪10%
suefile = open("sue.pkl", "wb")
pickle.dump(sue, suefile)
suefile.close()

# 查看该记录是否更新成功
suefile = open("sue.pkl", "rb")
print("sue.pkl['pay']", "=>\n", pickle.load(suefile)["pay"])
suefile.close()

sue.pkl['pay'] =>
 44000.0


上述方法数据库的中的key变为了实际的文件名。也就是说文件系统成为了高层字典，通过文件名可以直接访问每条记录

#### shelves 
自动地将对象pickle进和pickle出健访问文件系统。它们很像必须打开着的字典，存储持久化对象的字典 

In [24]:
# 创建数据库
import shelve
db = shelve.open("people-shelve") 
db["bob"] = bob
db["sue"] = sue
db.close()

In [25]:
# 访问数据库, 依旧是字典的字典
db = shelve.open("people-shelve")
for key in db:
    print(key, "=>\n", db[key])

bob =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 44000.0, 'job': 'hdw'}


In [27]:
# 更新
tom= dict(name="Tom", age=50, job=None, pay=0)

db = shelve.open("people-shelve")
sue = db["sue"]
sue["pay"] *= 1.1
# 手动写回是缺省需求，可以在open中定义关闭shelve时写回，但是会消耗内存使得关闭变慢
# 缺省是需要时创建，否则以可读可写方式打开 
db["sue"] = sue   
db["tom"] = tom
db.close() # 关闭shelve、关闭缓存

# 可以将数据库操作单独制作成脚本文件，通过运行相应程序 自动对本地数据库操作并将变化状态持久化地保存在文件中，

#### OOP

In [29]:
# 实例化类的方式来实现记录数据库
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job

In [31]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")
bob.name.split()[-1] # 获取姓氏

'Smith'

In [32]:
# 涨薪
sue.pay *= 1.1
sue.pay

38500.0

In [33]:
# 将类的实例放入列表和字典，构成一个整体
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

people = [bob, sue]  # 构成一个数据库列表
for person in people:
    print(person.name, person.pay)

Bob Smith 30000
Sue Jones 35000


In [34]:
x = [(person.name, person.age, person.pay,  person.job) for person in people]
x

[('Bob Smith', 42, 30000, 'software'), ('Sue Jones', 45, 35000, 'hartware')]

In [36]:
# SQL 风格查询
[rec.name for rec in people if rec.age >= 45]

['Sue Jones']

In [37]:
#  为类添加行为
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    
    def get_LastName(self):
        return self.name.split()[-1]

    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)

In [38]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

bob.get_LastName()

'Smith'

In [39]:
print(sue.pay)
sue.giveRaise()
print(sue.pay)

35000
38500.0


In [41]:
# 继承  通过人员类型编写子类
class Manager(Person):
    
    # 比如管理层除了正常涨薪还有 额外百分比的奖金的话
    # 重写该功能
    def giveRaise(self, percent=0.1, bonus=0.1) :
        self.pay *= (1.0 + percent +bonus)

In [42]:
tom = Manager(name="Tom Doe", age=50, pay=50000)
tom.get_LastName() # 父类的功能

'Doe'

In [54]:
tom.giveRaise(bonus=.20) # 自定义的涨薪功能
tom.pay

1075296.2364222005

#### 重构

In [ ]:
# 增强方法
class Manager(Person):
    
    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent+bonus)
# 1.调用父类的方法显式地传入self参数
# 2. 虽然重新定义了该方法，但我们调用的是通用的方法，不但可以降低冗余（业务逻辑只出现一次）
# 3.实践中易于启动父类的构造器

In [80]:
# 格式化显示
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    # 操作符重载
    def __str__(self):
        # 自定义返回值,对当对象需要作为整体打印时，比实例缺省的显示好得多
        return "<%s => %s: %s, %s>" % (self.__class__.__name__, self.name, self.job, self.pay)

    def get_LastName(self):
        return self.name.split()[-1]
    
    # 可以编写 __add__ 使得 + 表达式自动调用 giveraise方法，代价是代码可读性变差，不知到具体你 + 了啥？
    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)


class Manager(Person):
    
    def __init__(self, name, age, pay):
        Person.__init__(self, name, age, pay=0, job="Manager")

    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent+bonus)

In [81]:
bob = Person("Bob Smith", 44)
sue = Person("Sue Jones", 47, 40000, "Hardware")
tom = Manager(name="Tom Doe", age=50, pay=50000)
for obj in (bob, sue, tom):
    obj.giveRaise(.10)
    print(obj)

<Person => Bob Smith: None, 0.0>
<Person => Sue Jones: Hardware, 44000.0>
<Manager => Tom Doe: Manager, 0.0>


In [82]:
# 增加持久化
bob = Person("Bob Smith", 44)
sue = Person("Sue Jones", 47, 40000, "Hardware")
tom = Manager(name="Tom Doe", age=50, pay=50000)

# 创建了类实例的shelve、然后赋给新建的shelve文件的键
db = shelve.open("class-shelve")
db["bob"] = bob
db["sue"] = sue
db["tom"] = tom
db.close()

In [83]:
# 检验数据库
db = shelve.open("class-shelve")
for key in db:
    print(key, " =>\n ", db[key].name, db[key].pay)

bob  =>
  Bob Smith 0
sue  =>
  Sue Jones 40000
tom  =>
  Tom Doe 0


In [85]:
bob = db["bob"]
print(bob.get_LastName())  # 不需要导入类相关类，会自动定位相关类

Smith


In [90]:
from tkinter import * 
Label(text="Spam").pack()
mainloop()